In [2]:
import json
import os
from copy import deepcopy
from pathlib import Path

import dspy
import pandas as pd
import typer
import weave
from dotenv import load_dotenv
from dspy.evaluate import Evaluate
from dspy.teleprompt.ensemble import Ensemble
from rich.console import Console

from mhqa.agent import make_decomposing_agent, make_simple_agent
from mhqa.evaluation import (
    aggregate_scores,
    compute_scores,
    compute_scores_dataframe,
)
from mhqa.multihop import make_multihop_program
from mhqa.qa import make_qa_program
from mhqa.search import make_retriever
from mhqa.utils import configure_lm, dynamic_import, set_seed
import weave

print = Console(stderr=True).print

load_dotenv()

set_seed(89)

/home/baris/repos/pipeline-mhqa-dspy/.venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [3]:
weave.init(project_name="mhqa-dspy-debug")


def preprocess_examples(examples: list[dspy.Example], technique: str):
    if "agent" in technique or "multihop" in technique:
        return [example.with_inputs("docs", "question") for example in examples]
    else:
        return [example.with_inputs("context", "question") for example in examples]


def make_program(technique: str, retriever_name: str, top_k: int):
    retriever = make_retriever(retriever_name, top_k=top_k)
    if technique == "agent-simple":
        return make_simple_agent(retriever)
    elif technique == "agent-decompose":
        return make_decomposing_agent(retriever)
    elif technique == "multihop-decompose":
        return make_multihop_program(retriever)
    else:
        return make_qa_program(technique)


@weave.op()
def evaluate_answer(example, pred, trace=None):
    scores = compute_scores(pred.answer, example.answers)
    return scores["f1"]


def make_optimizer(optimizer_config: dict):
    cls = dynamic_import("dspy.teleprompt", optimizer_config["class"])
    kwargs = deepcopy(optimizer_config["params"])
    if optimizer_config["with_metric"]:
        kwargs["metric"] = evaluate_answer
    return cls(**kwargs)


def preprocess_result(result):
    example, pred, score = result
    predictions = {f"predicted_{k}": v for k, v in dict(pred).items()}
    return {**dict(example), **predictions, "score": float(score)}


def make_results_dataframe(results):
    dataf = pd.json_normalize([preprocess_result(result) for result in results])
    dataf["n_hops"] = dataf["question_decomposition"].apply(len)
    dataf["predicted_answer"] = dataf["predicted_answer"].fillna("No Answer")
    return compute_scores_dataframe(dataf)




Logged in as Weights & Biases user: bdsaglam.
View Weave data at https://wandb.ai/bdsaglam/mhqa-dspy-debug/weave


In [6]:
# Hardcoded values based on params.yaml

# Create necessary Python objects with hardcoded values
dataset_path = "bdsaglam/musique-sweep"
dataset_name = "answerable"
dataset_split = "train"

optimizer = "noop"
ensemble = "no"

evaluation_dataset_path = "bdsaglam/musique-sweep"
evaluation_dataset_name = "answerable"
evaluation_dataset_split = "validation"

retriever = "t5"
top_k = 3

model = "llama-3.3-70b"
temperature = 0.1
technique = "multihop-decompose"

load_from = "UNSET"
out = Path("output_directory")  # Placeholder for output directory

In [5]:
# Set up LLM
configure_lm(model, temperature)

# Load and preprocess datasets
if "musique" in dataset_path:
    from mhqa.musique import load_examples

    examples = preprocess_examples(load_examples(dataset_path, dataset_name, dataset_split), technique)[:10]
    print(f"Loaded {len(examples)} examples")
else:
    raise ValueError(f"Unknown dataset: {dataset_path}")

# Create the program
program = make_program(technique=technique, retriever_name=retriever, top_k=top_k)
if load_from and load_from != "UNSET":
    print(f"Loading model from {load_from}")
    program.load(load_from)

# Evaluate the program
evaluate_program = Evaluate(
    metric=evaluate_answer,
    devset=examples,
    num_threads=1,
    display_progress=True,
    return_outputs=True,
)
_, results = evaluate_program(program)

Loaded 10 examples

Loading default t5 model for language en
Default Model: unicamp-dl/InRanker-base
Loading T5Ranker model unicamp-dl/InRanker-base (this message can be suppressed by setting verbose=0)
No device set
Using device cpu
No dtype set
Using dtype torch.float32
Loading model unicamp-dl/InRanker-base, this might take a while...
Using device cpu.
Using dtype torch.float32.


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


T5 true token set to ▁true
T5 false token set to ▁false
Returning normalised scores...
Inputs template set to Query: {query} Document: {text} Relevant:
  0%|          | 0/10 [00:00<?, ?it/s]

Scoring...:   0%|          | 0/1 [00:00<?, ?it/s]

...

Scoring...:   0%|          | 0/1 [00:00<?, ?it/s]

Scoring...:   0%|          | 0/1 [00:00<?, ?it/s]

🍩 https://wandb.ai/bdsaglam/mhqa-dspy-debug/r/call/0194fde2-76d2-7ab3-ae9b-ad6c1bd781be
🍩 https://wandb.ai/bdsaglam/mhqa-dspy-debug/r/call/0194fde2-b35c-7540-86f7-a90cd0a8c889
Average Metric: 4.52 / 10 (45.2%): 100%|██████████| 10/10 [01:32<00:00,  9.26s/it]

2025/02/13 08:58:00 INFO dspy.evaluate.evaluate: Average Metric: 4.515151515151515 / 10 (45.2%)


TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [ ]:
out.mkdir(parents=True, exist_ok=True)

# Save the results
result_df = make_results_dataframe(results)
result_df.to_json(out / "results.jsonl", orient="records", lines=True)

# Save the scores
scores = aggregate_scores(result_df)
for n_hops in result_df["n_hops"].unique():
    scores[f"{n_hops}hops"] = aggregate_scores(result_df[result_df["n_hops"] == n_hops])

with open(out / "scores.json", "w") as f:
    json.dump(scores, f, indent=2)
